# Squad404

In [249]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, VGG16
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import StratifiedKFold




### Loading the augmented images and converting to pd dataframe

In [250]:
# Reading the csv and images 
data_annotations_csv = 'augmented_images/image_labels.csv'
augmented_images = 'augmented_images'

data_file = pd.read_csv(data_annotations_csv, dtype={'label': str})


In [251]:
# Creating vectors of filenames and labels matched by index 
img_filenames_vector = data_file['filename'].values
labels_vector = data_file['label'].values

### Creating our model

Add prototypical network to our model before running it through the training

In [252]:
# Function to create a model from the pretrained CNN base you pass in
# Example usage: model = create_model(MobileNetv2)
def create_model(base):
    base_model = base(input_shape=(224, 224, 3), include_top=False)
    base_model.trainable = False # Freeze the base_model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


### Setting up k-fold

In [253]:
k = 5
kf = StratifiedKFold(n_splits=k, shuffle=True)

In [254]:
for train_index, val_index in kf.split(img_filenames_vector, labels_vector):
    print(f"train: {train_index}, val: {val_index}")

train: [ 0  2  3  4  5  6  7  8 10 11 12 13 14 15 17 19 20 22 23 24 25 26 27 28
 29 30 31 32 33 34 35 36 37 38 39 41 42 43 44 45 46 47 50 51 53 54 55 56
 57 58 59 60 62 63 66 67 69 70 71 72 74 75 76 77 79 80 82 83 85 86 87 88
 90 92 94 95], val: [ 1  9 16 18 21 40 48 49 52 61 64 65 68 73 78 81 84 89 91 93]
train: [ 1  2  3  4  6  7  9 10 12 13 16 17 18 19 20 21 23 24 26 27 30 33 34 35
 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 58 59 60
 61 63 64 65 66 68 69 70 71 72 73 74 75 76 77 78 79 80 81 84 85 86 87 88
 89 91 92 93 95], val: [ 0  5  8 11 14 15 22 25 28 29 31 32 57 62 67 82 83 90 94]
train: [ 0  1  2  3  4  5  7  8  9 10 11 13 14 15 16 18 19 21 22 23 24 25 26 27
 28 29 30 31 32 35 36 39 40 41 43 46 47 48 49 50 51 52 53 55 56 57 58 59
 61 62 63 64 65 66 67 68 70 72 73 74 75 76 77 78 79 80 81 82 83 84 85 89
 90 91 93 94 95], val: [ 6 12 17 20 33 34 37 38 42 44 45 54 60 69 71 86 87 88 92]
train: [ 0  1  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 21 22 23 24 25 28

In [255]:
CNN_scores = []
CNN_models_to_test = [MobileNetV2, ResNet50]

In [256]:
for CNN_base_model in CNN_models_to_test:
    fold_scores_for_CNN = []
    fold_no = 1
    print(f"TRAINING FOR {CNN_base_model}")
    for train_index, val_index in kf.split(img_filenames_vector, labels_vector):
        print(f"Fold number: {fold_no}/5")
        train_filenames, val_filenames = img_filenames_vector[train_index], img_filenames_vector[val_index]
        train_labels, val_labels = labels_vector[train_index], labels_vector[val_index]
        
        # Create ImageDataGenerator for train and validation
        train_datagen = ImageDataGenerator(rescale=1./255)
        val_datagen = ImageDataGenerator(rescale=1./255)
        
        # Create generators
        train_generator = train_datagen.flow_from_dataframe(
            dataframe=data_file.iloc[train_index],
            directory=augmented_images,
            x_col='filename',
            y_col='label',
            target_size=(224, 224),
            batch_size=16,
            class_mode='binary'
        )
        
        val_generator = val_datagen.flow_from_dataframe(
            dataframe=data_file.iloc[val_index],
            directory=augmented_images,
            x_col='filename',
            y_col='label',
            target_size=(224, 224),
            batch_size=16,
            class_mode='binary'
        )
        model = create_model(CNN_base_model)
        history = model.fit(train_generator, validation_data=val_generator, epochs=5)
        fold_scores_for_CNN.append(model.evaluate(val_generator))
        fold_no += 1
    CNN_scores.append(fold_scores_for_CNN)

TRAINING FOR <function MobileNetV2 at 0x29fabaca0>
Fold number: 1/5
Found 76 validated image filenames belonging to 2 classes.
Found 20 validated image filenames belonging to 2 classes.
Epoch 1/5


/Users/lukewang/Documents/COMP9417/Squad404/venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 235ms/step - accuracy: 0.5256 - loss: 1.6556 - val_accuracy: 0.6000 - val_loss: 1.7810
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.7092 - loss: 1.1855 - val_accuracy: 0.6500 - val_loss: 0.4931
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.8032 - loss: 0.3744 - val_accuracy: 0.9000 - val_loss: 0.2674
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.9690 - loss: 0.1279 - val_accuracy: 0.8000 - val_loss: 0.3061
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - accuracy: 0.9235 - loss: 0.1288 - val_accuracy: 0.8500 - val_loss: 0.4212
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8375 - loss: 0.2895 
Fold number: 2/5
Found 77 validated image filenames belonging to 2 classes.
Found 19 validated image filenames belonging to 2 classes.
Epoch 1/5


/Users/lukewang/Documents/COMP9417/Squad404/venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 222ms/step - accuracy: 0.5902 - loss: 1.3221 - val_accuracy: 0.5789 - val_loss: 0.8173
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.5941 - loss: 0.8045 - val_accuracy: 0.6316 - val_loss: 1.0448
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.9003 - loss: 0.4025 - val_accuracy: 0.8947 - val_loss: 0.4164
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.9514 - loss: 0.1708 - val_accuracy: 0.8947 - val_loss: 0.2989
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.9957 - loss: 0.0756 - val_accuracy: 0.8421 - val_loss: 0.2994
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8322 - loss: 0.2934 
Fold number: 3/5
Found 77 validated image filenames belonging to 2 classes.
Found 19 validated image filenames belonging to 2 classes.
Epoch 1/5


...

5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 587ms/step - accuracy: 0.4670 - loss: 1.0996 - val_accuracy: 0.5263 - val_loss: 1.0981
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 449ms/step - accuracy: 0.5197 - loss: 0.7728 - val_accuracy: 0.4737 - val_loss: 0.8819
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 440ms/step - accuracy: 0.5596 - loss: 0.7486 - val_accuracy: 0.5789 - val_loss: 0.5460
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 435ms/step - accuracy: 0.6019 - loss: 0.6180 - val_accuracy: 0.5789 - val_loss: 0.8504
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 450ms/step - accuracy: 0.6318 - loss: 0.6359 - val_accuracy: 0.5789 - val_loss: 0.4862
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.5735 - loss: 0.6440
Fold number: 4/5
Found 77 validated image filenames belonging to 2 classes.
Found 19 validated image filenames belonging to 2 classes.
Epoch 1/5


/Users/lukewang/Documents/COMP9417/Squad404/venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 592ms/step - accuracy: 0.6046 - loss: 0.7277 - val_accuracy: 0.5789 - val_loss: 0.7954
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 433ms/step - accuracy: 0.5470 - loss: 0.7464 - val_accuracy: 0.5263 - val_loss: 0.6222
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 433ms/step - accuracy: 0.5559 - loss: 0.6732 - val_accuracy: 0.6842 - val_loss: 0.6454
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 431ms/step - accuracy: 0.6679 - loss: 0.6015 - val_accuracy: 0.6316 - val_loss: 0.5631
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 451ms/step - accuracy: 0.7279 - loss: 0.5883 - val_accuracy: 0.6842 - val_loss: 0.4605
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.6853 - loss: 0.5261
Fold number: 5/5
Found 77 validated image filenames belonging to 2 classes.
Found 19 validated image filenames belonging to 2 classes.
Epoch 1/5


/Users/lukewang/Documents/COMP9417/Squad404/venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 590ms/step - accuracy: 0.5588 - loss: 1.0659 - val_accuracy: 0.5263 - val_loss: 0.9593
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 434ms/step - accuracy: 0.5046 - loss: 0.8212 - val_accuracy: 0.6316 - val_loss: 0.6431
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 435ms/step - accuracy: 0.5976 - loss: 0.7087 - val_accuracy: 0.5263 - val_loss: 0.5998
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 432ms/step - accuracy: 0.6011 - loss: 0.6981 - val_accuracy: 0.7368 - val_loss: 0.9237
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 436ms/step - accuracy: 0.6126 - loss: 0.6499 - val_accuracy: 0.4737 - val_loss: 1.0820
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.4616 - loss: 0.8448


In [257]:
for score in CNN_scores:
    print(np.mean(score, axis=0))

[0.1614833 0.8963158]
[0.57276335 0.76052632]
[0.65637482 0.59421054]
